# Background

In this case study, we will develop a model to predict spatial variations in **annual average** **outdoor ultrafine particle (UFP) concentrations** across the Bucaramanga metropolitan area in Colombia. We are reproducing work that was published here: https://pubs.acs.org/doi/10.1021/acs.est.1c01412
  
**Setting**  
The Bucaramanga metropolitan area consists of the following cities: Bucaramanga, Floridablanca, Giron, and Piedecuesta. It has a population of approximately 1.3 million and has been growing rapidly in recent years. There are some databases available with limited land use information, but not nearly as much as we would find for a large city in a high income country. That makes developing a land use regression model a challenge, but it is an opportunity to use alternative data streams such as digital images to train deep convolutional neural network models. We will train our models on monitoring data and either satellite or street-view images.    

**Monitoring Data**  
During the monitoring campaing, UFP concentrations were repeatedly measured at 1-second intervals along roads throughout study area during a 2-week period. The road network was divided into 100 m long road segments and the median of all UFP measurements along a given road segment was assigned to the centroid (i.e. middle) of the road segment as the estimate of the annual average. The number of UFP measurements along a given road segment is reffered to as the "join_count" in this data (i.e., how many 1-second points were joined to the centroid of the road segment). The mean meteorological conditions at the nearest airport during monitoring were measured and linked to the aggregated monitoring data. This was done to account for weather-related temporal variations in UFP concentrations during monitoring (additional details below).  
Black Carbon (BC) concentrations were also measured during the monitoring campaign and those data have been included in case you would like to try modelling BC spatial variations on your own.
  
**Satellite Images**  
Two satellite-view images were downloaded for each road segment using the *googleway* package in R. The images were centered on each road segment and one was at a zoom level of 18 and the other was at a zoom level of 19. Picking the zoom level is a bit of an art. A smaller zoom level will have a lot of overlap between images (e.g., zoom 0 is the whole world and all the images would be the same!) A large zoom level would lose some of the contextual information around the road segment. You can play around in google maps to get a feel for the spatial coverage and trade-offs between zoom levels.

**Street Images**  
Two street-view images were downloaded for each road segment using the *googleway* package as well. The images were oriented to look up and down the street.
  
**Temporal Variations**
This mobile monitoring campiagn used a single monitor in a vehicle that drove around the study area. Roads were visited multiple times in an effort to get a a sample of measurements that representated the annual average, but the road segments were still visited a different times of day with different weather conditions. To account for this, we will later include meteorological conditions during monitoring with the CNN predictions in a regression in order to help control for the weather-related temporal variations in the monitoring data. This will be done after training the CNN models.

Additional background information can be found in the published article: https://pubs.acs.org/doi/10.1021/acs.est.1c01412 (same link as above).

# Set Runtime (IMPORTANT!)

Goolge Colab offers free acess to hardware upgrades such as a Graphics Processing Unit (GPU). That will allow us to train our deep learning models in a reasonable amount of time. If you try to train using a CPU, it will take a very ***very*** long time. Here's a quick blog post explaining why: https://towardsdatascience.com/what-is-a-gpu-and-do-you-need-one-in-deep-learning-718b9597aa0d


***Change to a GPU using the drop down menu***
> Runtime > Change runtime type > GPU > OK

This will restart your runtime, which removes any data and variables that you have read in. If you don't change to a GPU here, then you would end up losing all the prepared data and have to redo it.

After you have changed the runtime, run the code below to verify that you are indeed connected to a GPU.


In [ ]:
# this code checks to see if you have a GPU available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found, make sure to connect to GPU') # if not avaible, you will get an error
print('Good to go! Found GPU at: {}'.format(device_name))

Good to go! Found GPU at: /device:GPU:0


# FYI - Some Google Colab Information and Tips

We've shared a workbook called "Python and Colab Intro" that can help show you some basics. Additionally:  

Google Colab FAQs
https://research.google.com/colaboratory/faq.html#:~:text=Runtimes%20will%20time%20out%20if,on%20your%20compute%20unit%20balance.

Google Colab tips:  
https://www.analyticsvidhya.com/blog/2021/05/10-colab-tips-and-hacks-for-efficient-use-of-it/

There are keyboard shortcuts/commands that can be useful, but it can take a bit of getting used to. If you are new to Google Colab, don't bother with using them just yet. If you do want to try, push "Ctrl+M" ("⌘+M" on Mac), then release those keys and push "H" to show the keyboard shortcuts.  
Some useful commands:
>  "⌘/Ctrl+M then H": show keyboard shortcuts (this also allows you to set them)  
>  "⌘/Ctrl+M then -": splits cell where the cursor is  
>  "⌘/Ctrl+M then B": insert code cell below

# Importing Modules/Packages

Like in R, there are several packages that we want to load/import. If you want to be pedantic, some are technically "modules", some are "submodules" and others are "packages". For our purposes it doesn't matter and we'll just call them all packages here.  

Time to load our packages. We are also going to define Root Mean Squared Error (RMSE). We will need that later when we are training models on continuous values of UFP concentrations. The Keras package has Mean Absolute Error (MAE), but not RMSE. We prefer using RMSE as the loss function because it gives a greater penalty to large errors than MAE.





In [ ]:
#########################################################################################################
##################### Run this cell whenever you connect to a new runtime ###############################
#########################################################################################################

# this step loads the packages we will be using

# deep learning and data science packages
import tensorflow as tf # if your instance of google colab can't load tensorflow, close and re-open
from tensorflow import keras as K
import pandas as pd
import numpy as np

# for file processing and moving
import os
import zipfile
import shutil

# for figures
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

# for linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# for the Grad-CAMs
from IPython.display import Image, display
import matplotlib.cm as cm

# define root mean square error (rmse) for use as the loss function when training models on continuous UFP. mae is also available
def rmse(y_true, y_pred):
    return K.backend.sqrt(K.backend.mean(K.backend.square(y_pred - y_true)))



# Loading Data and Setting Up Folders

There are several methods you can use to access the data in your Google Drive.

> ***We will run code to mount entire drive, then read in files***

Colab disconnect your runtime if you sit idle too long. When you reconnect, you have to re-read in the data. For that reason, we like to mount the drive using code. It is also possible to mount the drive by pointing and clicking on the folder in the tab to the left and then clicking the Google Drive icon (folder with triangle on it).  

Run the code below to mount the drive. You will get a pop up that will then open a new tab. Agree to both.   

> ***This can take about 10 - 60 seconds to run***


In [ ]:
#########################################################################################################
##################### Run this cell whenever you connect to a new runtime ###############################
#########################################################################################################

# use this code to mount your drive. it may take a moment.
# a pop up and new tab will ask for permissions to read and write in your google drive
from google.colab import drive
drive.mount('/content/drive')

# # in case you want to unmount the drive using code
# drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After giving persmission, google drive will show up in the folders. You can click on the folder icon in the ribbon to the left to show/hide the folders. Your Google drive should be in "*/drive*" and "*/MyDrive*". Within "*/MyDrive*", you should be able to find the "*/sharp_ml_course*" folder that we shared with you.

> *If you can't find the "/sharp_ml_course" in the folders in the ribbon to the left, then you likely need to add the Google Drive folder we shared with you. Go to your Google Drive (drive.google.com), sign in, click on "Shared with me" on the left, find and right click on "sharp_ml_course", click on "Add shortcut to Drive", make sure "My Drive" is highlighted and then click "ADD SHORTCUT".*

FYI - For the "point and click" method to mounting the drive, you click on the folder in the ribbon to show folders, then you click on the folder with the triangle (i.e., google drive symbol) and then click the refresh folder icon.



**Unzip Images**

We'll look at the metadata in a moment, but first we should unzip the images into a working folder. We'll start by creating a folder called "images" to unzip the images into. This will just be a working folder for this session. If the runtime restarts, then we'll have to unzip again, but that's actually kind of handy. It's a pain to have tons of unzipped images in a folder - slow to navigate through, slow to copy, slow to move. It's just easier to unzip them with code when we need them.


> ***Unzipping all the images will take about 30 - 60 seconds***



In [ ]:
#########################################################################################################
##################### Run this cell whenever you connect to a new runtime ###############################
#########################################################################################################

# if the directory (i.e., folder) doesn't already exist, make the folder
if not os.path.exists('/content/images'):
  !mkdir /content/images

# this will take about 30 - 60 seconds
with zipfile.ZipFile("/content/drive/MyDrive/sharp_ml_course/colombia_sat_street_images/colombia_sat_street_images.zip","r") as zip_ref:
    zip_ref.extractall("/content/images/")


Once this is done you will see the subfolders. Take a moment to see how the folders are organized. In this session, you should avoid expanding the images subfolders (e.g., images_18); they have thousands of images and it's slow to show all of them.
> images
>> images_18 *(zoom 18 satellite images)*  
>> images_19 *(zoom 19 satellite images)*  
>> images_angle1 *(street images looking either up or down the street)*  
>> images_angle2 *(street images looking in the other direction)*  

**Read in Metadata**

In the folder ribbon on the left, go to the "*/sharp_ml_course*" folder. You can right click on the "*colombia_image_ghp7_metadata.csv*" file to copy the file path. The filepath in the code cell below should be correct.

In [ ]:
# read in the metadata. if this code doesn't work, verify the path to the csv file
metadata = pd.read_csv("/content/drive/MyDrive/sharp_ml_course/colombia_image_ghp7_metadata_sharp2023.csv")



# Directories (Folders) for Outputs

For our purposes here, we use the terms "directories" and "folders" interchangably.  

Let's create some folders to save our outputs. In this case, we want to save them to our Colab Notebooks folder in the google drive. It takes a long time to train these models, so we want to make sure we keep them even if the session restarts. If we saved them in */content* then we would lose them when we disconnected from our runtime.   

Notice that we have to make parent directories first. You'll get an error if you try to create a directories that already exists.

In [ ]:
# if the directories don't exist, make them. only evaluating the top directory
if not os.path.exists('/content/drive/MyDrive/colab_files'):
  !mkdir /content/drive/MyDrive/colab_files
if not os.path.exists('/content/drive/MyDrive/colab_files/sharp_d2_UFP'):
  !mkdir /content/drive/MyDrive/colab_files/sharp_d2_UFP
  !mkdir /content/drive/MyDrive/colab_files/sharp_d2_UFP/output
  !mkdir /content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_log
  !mkdir /content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model
  !mkdir /content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_predictions
  !mkdir /content/drive/MyDrive/colab_files/sharp_d2_UFP/output/grad_cam_images

# Explore the Metadata

Now let's take a look at the metadata. This will show the monitoring data and file names of the associated images. We'll explain the variables further down.

In [ ]:
# this will print out the metadata variable
metadata



**Metadata Columns**  
*id*: id number we have given to road segments  
*join_count*: mount of monitoring time (seconds) on a given road segment  
*long*: longitude, WSG 84  
*lat*: latitude, WSG 84  
*road_angle*: road angle was used to download street view images to make sure they were looking up and down the street  
*geohash*: this indicates what gridsquare the road segment is in, we use this to split the data in order to minimize the overlap between training and test sets  
*file*: name of the image file, notice how the number is the same as the id  
*ufp*: ultrafine particle concentration (pt/cm3)  
*ufp_cat*: quitile of UFP concentration   
*bc*: black carbon concentration (ng/m3)  
*bc_cat*: quitile of BC concentration  
*temp_f*: mean tempreature during monitoring (deg F)  
*relhum_pct*: mean relative humidity during monitoring (%)  
*winddir_deg*: mean wind direction during monitoring (degrees)  
*windspeed_kts*: mean windspeed during monitoring (knots)  


**Strings vs Numeric**  

As humans, we know that *3*, *three*, and *\"3"* all mean the same thing. Python is not human and will treat those differently. The code below shows you the data type of each column in the metadata. Note that the dtype for columns of strings is "object" (the pandas module does this for efficiency reasons).  

When we read in the metadata, Python recognized that the *ufp_cat* column was a bunch of numbers, specifically integers. That's great if we want to treat the UFP quintiles as numeric values, but if we want to treat it as categories, we will have to change it to string. It's as though we are changing *3* to *three*.  

Using the wrong data type is an easy way to get tripped up. One of the first things you should do when troubleshooting code is check all the variable data types and make sure they are appropriate.   

More reading on data types:  
https://www.geeksforgeeks.org/python-data-types/


In [ ]:
# .dtypes at the end of a variable will print out the data types for each column
metadata.dtypes


**Exploratory Plots**

Recall that *join_count* is the amount of monitoring time on a given road segment. The histogram below shows the distribution of monitoring time per road segment.  


In [ ]:
# using 'matplotlib' package (plt.),  plot a histogram of 'join_cout'
plt.hist(x= metadata['join_count'], bins='auto', color='#0504aa', rwidth=0.85) # rwidth=0.85 adds a bit of space between columns
plt.xlabel('Road Segement Monitoring Time (s)')
plt.ylabel('Frequency')
plt.title('Histogram of Road Segment Monitoring Times')
plt.xlim(xmin = -1, xmax = 200)  # limit to 200 so it is easier to visualize




We probably don't want to keep road segments that only have 1 second of monitoring time (does 1 second of monitoring represent the annual average?!?), but we also don't want to be too strict and lose too much spatial coverage. If we insist on a minimum of 120 seconds per road segment, we'll have much less spatial coverage. Run the code below to see the spatial coverage when restricting to minimum 120 seconds vs 10 seconds per road segment. You can check out the Bucaramanga metropolitan area in Google Maps for reference:  
https://goo.gl/maps/oKmdRzAZjDp8fhKw8


In the end, we decided to use 10 seconds as our threshold. You could probably justify using other thresholds as well (e.g., 20 seconds). You can take a look at the supplementary material of the published article and see our results when using different thresholds.

***Try later:*** *You can try developing models using different thresholds. What happens to model R2? What happens to spatial coverage?*

In [ ]:
# to create two plots side by side, we define the figure and the two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(10, 7))

# then we fill the subplots (notice ax = ax1).
# Here we are using the 'seaborn' (sns.) package for plots because I like the defaults for scatter plots
sns.scatterplot(x='long', y='lat', data=metadata.loc[metadata['join_count']> 120], s = 10, ax = ax1).set(title='Road Segments with \n>120s of Monitoring')
sns.scatterplot(x='long', y='lat', data=metadata.loc[metadata['join_count']> 10], s = 10, ax = ax2).set(title='Road Segments with \n>10s of Monitoring')

# note that these are just scatter plots of longitude and latitute. they are somewhat distorted because they are not true spatial plots. this is fine for exploratory purposes

Let's also look at histograms of the measured UFP and BC concentrations. This is useful because if the distribution is heavily skewed, we may chose to log-transform it. We can see from the plots below that only BC is skewed.  

In [ ]:
# here is a different approach to plotting figures side by side.
f = plt.figure(figsize=(12,5)) # set figure size
ax = f.add_subplot(121) # sets up first plot, (121) sets up the position https://stackoverflow.com/questions/3584805/what-does-the-argument-mean-in-fig-add-subplot111
ax2 = f.add_subplot(122) # sets up second plot

# fill in the two plots with histograms
ax.hist(x= metadata.loc[metadata['join_count'] >= 10, 'ufp'], bins='auto', color='purple',rwidth=0.85)
ax2.hist(x= metadata.loc[metadata['join_count'] >= 10, 'bc'], bins='auto', color='grey', rwidth=0.85)

# set labels, titles, and limits for each plot
ax.set_xlabel('UFP Concentration (pt/cm3)') # upadate label to quint vs continous
ax2.set_xlabel('BC Concentration (ng/m3)')
ax.set_ylabel('Frequency')
ax.set_title('Observed UFP Concentrations (min 10 s/road segment)')
ax2.set_title('Observed BC Concentrations (min 10 s/road segment)')
ax2.set_xlim(0, 120000) # set the axis limits on the BC plot to help with the visual. I did this knowing what it looked like.

**Geohash Codes**  

A geohash code is a unique alpha-numeric code for gridsquares anywhere on earth. You can think of it as a way to spatially group our road segments together. This will be useful for when we split our data into train, validate, and test sets. The quick scatter plot below shows a section of the study area and the colors represent different geohash gridsquares. Notice how the colors are grouped together.         

More information on geohash codes can be found here:  
http://ellse.org/uncategorized/how-geohash-works/  

https://www.movable-type.co.uk/scripts/geohash.html  

In [ ]:
# make a subset of the data that covers a small area. it would be cluttered and confusing if we plotted all the data
# data.loc[(condition) & (condition)] is used to filter by multiple conditions
sub_metadata = metadata.loc[(metadata['lat'] < 7.163) & (metadata['lat']> 7.155) & (metadata['long'] < -73.128) & (metadata['long'] > -73.138)]

# use seaborn to create a scatter plot
sns.scatterplot(x='long', y='lat', data= sub_metadata, hue='geohash', legend = False)
# note that this is just a scatter plot of longitude and latitute. it is distorted because it is not a true spatial plot. this is fine for exploratory purposes

**Instrument Limit of Detection**

The BC monitor was a microAeth MA200, which has a lower limit of detection of 30 ng/m3.

The UFP monitor was a TSI CPC 3007, which has a lower limit of detection of 0 pt/cm3. It also has an upper limit of detection of 100,000 pt/cm3. Measurements over this limit are less accurate.  

We will impute observations that fall below the lower limit of detection with half the value of the lower limit of detection. We won't change any of the values above the upper limit of detect, but it's good to keep in mind that this is a limitation of the data we are working with.



In [ ]:
# inspect to see if there are any rows with BC less than 30 or UFP less than zero
# | is "or"
metadata.loc[(metadata['bc'] < 30) | (metadata['ufp'] < 0)]

# Preparing the Metadata


Now we'll setup the metadata:  
1. Restrict the data to road segments with at least 10 seconds of monitoring;  
2. Impute any BC observations that are below the lower limit of detection (recall that there were no UFP observations below the limit of detection);
3. Log-transform the continuous BC column;  
4. Create categorical columns for the quintiles that have "string" values instead of numeric values. We'll keep the numeric value quintile columns in case we want to use them later;  
5. Split the data into train, test, and validate set. We'll split randomly by geohash code. We split by geohash code in order to spatially separate the sets, which will help reduce the overlap of the images in the sets. This makes them more independent and reduces "cheating".


*Try later: After this workshop, you could try splitting the data completely randomly (i.e., by "id" instead of "geohash") to see if it changes the model performance in the test set. Does a little cheating results in higher performance?*

In [ ]:
# we'll read the metadata in again. this ensures that we are always cleaning the raw data
# this also ensures that if we accidentally run this code twice, we'll still set up the metadata correctly
metadata = pd.read_csv("/content/drive/MyDrive/sharp_ml_course/colombia_image_ghp7_metadata_sharp2023.csv") # you could comment out this line of code and run this cell twice to see what happens

# restrict to road segements with at least 10 seconds of monitoring
metadata = metadata.loc[metadata['join_count'] >= 10] # you can replace 10 with a different value if you want to tray developing a model using a different threshold

# impute any BC values that are below the lower limit of detection (30) with half that value (15)
metadata.loc[metadata['bc'] < 30, 'bc'] = 15

# log transform bc
metadata['bc'] = np.log(metadata['bc'])

# the ufp_cat and bc_cat columns are numbers. we can either keep them numeric or categorical
# first give them more meaningful names, we'll include abreviations of quintile continuous
metadata = metadata.rename(columns={'ufp_cat': 'ufp_quint_cont', 'bc_cat': 'bc_quint_cont'}, errors = 'raise')

# create categorical columns that are strings.
# if we want to use a classification algorithm (i.e. softmax activation function), the outcome (i.e., pollutant we want to predict) will need to be strings
metadata[['ufp_quint_cat', 'bc_quint_cat']] = metadata[['ufp_quint_cont', 'bc_quint_cont']].astype(str)
metadata[['ufp', 'ufp_quint_cont', 'bc', 'bc_quint_cont']] = metadata[['ufp', 'ufp_quint_cont', 'bc', 'bc_quint_cont']].astype('float32')

# set seed and split metadata into train, val, and test
np.random.seed(1997)
train, validate, test = np.split(np.random.choice(pd.unique(metadata['geohash']), size = len(pd.unique(metadata['geohash'])), replace = False), [int(0.7*len(pd.unique(metadata['geohash']))), int(0.85*len(pd.unique(metadata['geohash'])))])
# np.random.choice puts the unique geohashs in a random order (replace = false means each unique geohash is sampled once) (there is no probability, so the default is each geohash has the same probability of being selected)
# np.split cuts the vector a specified locations, in this case the locations are 0.7 the length and 0.85 the length, this gives 0.7-0.15-0.15

metadata.loc[metadata['geohash'].isin(train), 'set'] = 'train'
# metadata['geohash'].isin(train) give a vector of T/F, it evaluates if the geohash row value in train
# metadata.loc[...] locates the rows based on the T/F vector
# ..., 'set'] = 'train' creates a new column 'set' and fills it with 'train', but only in the geohash that are in the train

metadata_nsites = metadata.shape[0] # we want to keep track of how many rows we have in the metadata. this will be useful down below

# inspect the metadata
metadata
# slide the scrolling bar of the output to the right, we can see that only the training set has been assigned

In [ ]:
# repeat for validate and test sets
metadata.loc[metadata['geohash'].isin(validate), 'set'] = 'validate'
metadata.loc[metadata['geohash'].isin(test), 'set'] = 'test'
metadata = metadata.sort_values(by='file')

metadata

**Check the Data Split**  

We can count how many rows are in each set. We expect it to be roughly 70%, 15%, and 15% in the train, validate, and test sets. It probably won't be exact because we randomly split by geohash code and each geohash code can have different numbers of road segments within it.  

In [ ]:
# count number of rows in each set
metadata.set.value_counts()

# Image Filepaths

The metadata has the image file names in it, but we should specify part of the filepaths as well. Each road segment has two satellite images (zoom 18 and 19) and two street view images (up and down the street).  

Recall the organization of the folders:  
> images/
>> images_18/  
>> images_19/  
>> images_angle1/  
>> images_angle2/  

Look back at output of the metadata. For site id = 1, the filename is "00001.jpg". Each image folder has a file named "00001.jpg". If you want one of the street-view images for site id = 1, you'll find it here:
> *images/images_angle1/00001.jpg*

The other street-view images for site id = 1 is here:
> *images/images_angle2/00001.jpg*

If you want the zoom 18 satellite image for site id = 1, you'll find it here:
> *images/images_18/00001.jpg*  

The zoom 18 satellite image for site id = 5911 is here:
> *images/images_18/05911.jpg*

You can see how changing the filepath is a way to select a specific image for a given site. We can create new column that points to the zoom 18 satellite images by pasting "images_18/" to the values in the file column.

If we are training on satellite images, we want to train on both zoom levels. To do so we'll create two copies of the metadata, assign one of the pictures to each copy in a new "sat_file" column, and then bind the rows of the two data sets.  

The same goes for street-view images, we want to train on images from both directions. We'll do the same except with the "images_angle1" and "images_angle2" in a new "str_file" column.   


In [ ]:
# create two copies of the metadata called s1 and s2
s1, s2 = metadata.copy(), metadata.copy()

# # define location of satellite images, zoom level 18 for one copy and zoom level 19 for the other
s1['sat_file'] = 'images_18/' + s1['file']  # create a column and fill it with the zoom 18 filepaths
s2['sat_file'] = 'images_19/' + s2['file']  # create a column with the same name and fill it with the zoom 19 filepaths

# # do the same for street images
s1['str_file'] = 'images_angle1/' + s1['file'] # create a column and fill it with the angle1 filepaths
s2['str_file'] = 'images_angle2/' + s2['file'] # create a column with the same name and fill it with the angle2 filepaths

# bind s1 and s2 together. now the metadata is twice as long as it was before
if metadata.shape[0] < metadata_nsites*2:  # if the metadata has fewer rows than 2 x the number of sites, then we bind s1 and s2. This makes sure that if we run this cell twice, we don't keep doubling the number of rows in the metadata
  metadata = pd.concat([s1, s2])

metadata

# Managing Images



Let's take a look as some images. It's neat to see all four images associated with a site, but more importantly, this is a good way to confirm that the image paths are correct. I have spent way too many hours troubleshooting a model that won't train only to realize that it was bad filepaths to images. When troubleshooting code, check data types and check file paths - those are easy mistakes but easy fixes!  

Recall the organization of the folders:  
> images/
>> images_18/  
>> images_19/  
>> images_angle1/  
>> images_angle2/  


In [ ]:
# take a look at the images
# this is will also confirm that the image_dir actually leads to the images, this is a good check that helps with troubleshooting

# the parent directory for images, we will use this later on in the code. we will use this as a "global variable", which will be explained in a text cell further below
image_dir = "/content/images/"

np.random.seed(seed = None) # unset the random seed so we get a random image
random_image_number = np.random.choice(metadata['file'])  # select a random image file name
np.random.seed(1997) # set the random seed back for reproducibility

# read in each type of image for the randomly selected site
# notice how the parent directory gets pasted with the type of image directory and then the image file name?
  # this is similar to what we did when creating the str_file and sat_file columns, we pasted file paths with the image file names
img_18 = mpimg.imread(image_dir + 'images_18/' + random_image_number)
img_19 = mpimg.imread(image_dir + 'images_19/' + random_image_number)
imgangle_1 = mpimg.imread(image_dir + 'images_angle1/' + random_image_number)
imgangle_2 = mpimg.imread(image_dir + 'images_angle2/' + random_image_number)

# set up the 4 subplots in 2x2 and plot show the images
f, axarr = plt.subplots(2, 2, figsize = (10,10))
axarr[0,0].imshow(img_18)
axarr[0,1].imshow(img_19)
axarr[1,0].imshow(imgangle_1)
axarr[1,1].imshow(imgangle_2)

# you can run this code chunk several times in order to see different pictures

In the plots above, notice the x and y axis values. Those are pixels. We have images with different resolutions. Keep that in mind as we work through the code.  

Here's another check we can do before we start training. It counts the number of files in a directory. We expect there to be the same number of files in each folder. We make a loop to count in each of the 4 sub directories with images.

In [ ]:
# subfolders we want to count the number of files in
sub_dirs = ['images_18/', 'images_19/', 'images_angle1/', 'images_angle2/']

for i in sub_dirs: # for each sub-directory, start the count at zero and then use the second loop to count all the files
  count = 0
  for path in os.listdir(image_dir + i):  # recall that we defined image_dir above when checking images, you want to use this every time because that means if you need to make a change, you are only changing in one spot!
      # check if current path is a file
      if os.path.isfile(os.path.join(image_dir + i, path)):
          count += 1  # += adds to and redefines the variable
  print(i, ' file count:', count)

So we are missing some files in *images_19*. Do you think we can still train a model even though we are missing 5 images? Absolutely. Might we get tripped up somewhere along the way? Also yes! Keras will automatically skip any invalid filenames (i.e., missing images), so you will be able to train the models and generate predictions. However you may run into problems when you try to incorporate the predictions into the metadata because you won't know which sites have missing images.

The code below identifies which files are in the *images_18* folder but not in the *images_19* folder.

In [ ]:
# list the files that are in each directory
list18 = os.listdir(image_dir + 'images_18/') # our global variable "image_dir" again!
list19 = os.listdir(image_dir + 'images_19/')

# which files in images_18 are not in images_19?
missing_19 = list(set(list18).difference(list19))
missing_19

Luckily, we have the missing files in *missing_images_19*. You can move them over to the correct folder.

If you want, you can go ahead and train the models without these missing images (i.e. don't run the code below and cary on) to see what happens.

In [ ]:
for i in missing_19:
  # renames the files, instead of changing the file's name, we change the directory
  os.rename(image_dir + 'missing_images_19/' + i, image_dir + 'images_19/' + i)

# # after moving the files, you can go back and check the counts in the folders
# # you can also try this:
# list(set(list19).difference(list18))

# Train Models

Now that the metadata is ready and we are happy with our images we can start setting up out models.

We start by definine our training and validation image data generators. These are functions that pre-process the data to prepare it for our CNN model. Here we are using *ImageDataGenerator().flow_from_dataframe()* which uses the metadata variable that we have set up. It expects the metadata to have a column indicating image filenames and a column indicating image labels (i.e., air pollution level). *ImageDataGenerator()* will resize images to a target size. Different model architectures have different acceptable model sizes. Xception's default is 299 x 299, but it is pretty flexible and can take anything larger than 150 x 150. EfficientNetB2 on the other hand requires 260 x 260. You can read more about these different architectures in the Keras documentation.     


Instead of using a dataframe input, we could have used *flow_from_directory()* which uses directories (i.e., folders) for each category of image. That way, the directory acts as a label for all the images within it. For us, that would require a folder for each quintile. Here is a tutorial on using *flow_from_directory()*:    
https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

In [ ]:
# define the training and validation generators
# now make one for each
generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=K.applications.xception.preprocess_input,
                                                      horizontal_flip=True,
                                                      vertical_flip = True) # set to false if using street images


train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='train', ['ufp', 'sat_file']].reset_index(drop=True),
                                                directory= image_dir,  # we already defined our image directory when we were ploting the pictures
                                                x_col= 'sat_file',  # this is the filename of the images
                                                y_col='ufp',  # this is the label for the images
                                                class_mode= 'raw',    # is the outcome continuous ('raw') or categorical
                                                target_size=(240, 240),  # all of our images will be resized to 240 x 240
                                                color_mode='rgb',  # this is the type of image. Note that some models don't work with grayscale
                                                batch_size=64,  # seems about the same speed with 32 or 64
                                                shuffle=True)

validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='validate', ['ufp', 'sat_file']].reset_index(drop=True),
                                                     directory= image_dir,
                                                     x_col= 'sat_file',
                                                     y_col='ufp',
                                                     class_mode='raw',  # is the outcome continuous ('raw') or categorical
                                                     target_size=(240, 240),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

# https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c



Notes on input size:  
https://datascience.stackexchange.com/questions/89860/input-shape-of-an-xception-cnn-model  
Several models are flexible on input size as long as it is over a certain minimum. If too small, the convolutions will transform the image data into nothing. The above post says Xception is probably best with 299x299 because that was the original set up, but you shouldn't resize images larger because you are not adding any new information and you're just adding more parameters (i.e. making it slower and larger) for no benefit. Other model architectures such as efficientNet requires a specific size.





**Change Control**

The code above is fine and works, but how many changes in the code would we have to make if we wanted to train on street-view images instead of satellite images? In the code above you would have to make five changes and there would be more changes in following code as well (e.g., model name when you are saving it). Instead of changing the code you could copy and paste it and have two different versions in different cells or maybe have different notebooks. Either way, if you make changes to your code, you would have to implement it in many spots. You can see how things might get out of control if you're making changes and how errors can insidiously sneak into your code.   


An easier way to handle this is to think of all the changes you might want to make (e.g., satellite vs street, UFP vs BC, etc.) and keep them in one place. This is similar to how we defined *image_dir* in one place and then kept using it in other code chunks.  A variable that impacts multiple functions and code chunks can be referred to as a "global variable". When figuring out if a variable will be "global" for you code, it is helpful to think of which changes might impact the code in multiple spots. For example, if we want to train on UFP categorical quintiles, then that impacts which column in the metadata is the *y_col*, but it also impacts the *class_mode* (*'categorical'* instead of *'raw'*). You can see how this can lead us to using some *if else* statements to set global variables. Without even thinking of how to code, this might be how you would explain the changes:


> if I want to predict UFP categorical quintiles:
>> -then I need the *y_col* to be *ufp_quint_cat*   
>> -I need *class_mode* to be set to *'categorical'*  
>> -I need to use a *'soft_max'* activation function  
>> -I care about prediction accuracy  
>> -I want to maximize prediction accuracy  

> but if I want to predict UFP continuous:
>> -then I need the *y_col* to be *ufp*   
>> -I need *class_mode* to be set to *'raw'*  
>> -I need a *'linear'* activation function  
>> -I care about mean prediction errors  
>> -I want to minimize prediction errors

Thinking through the problem sets us up to code a solution!

We find it can be helpful to combine the *input()* function prompts with some *if else* statements. That takes a little more planning and work up front, but it can make your life much easier in the long run and is a good approach for avoiding human errors.  

Below is some code that you can play around with to get comfortable with using *input()* with *if else* statements. It won't affect any other code.

In [ ]:
########## running the code will not affect any other code ##########
# this code shows you how the input function behaves
# you can play around with it

serious_answer = input("Are you having fun? ")  # this will open a prompt below, input your answer and press return/enter
print('Your answer was: '+ serious_answer) # this will print text with your answer

In [ ]:
########## running the code will not affect any other code ##########
# this code shows you how you can use the input with if else statements to set up your model
# you can play around with it

type_of_food = input("Do you prefer pizza or pasta? ")  # this will open a prompt below, input your answer and press return/enter

if type_of_food == 'pizza': # if you wrote pizza, then it will assign the value 'beer' to the beverage variable
  beverage = 'beer'
else:
  if 'past' in type_of_food: # if 'past' is in what you wrote, then it will assign the value 'wine' to the beverage variable. notice how this evaluation is more tolerant of typos
    beverage = 'wine'
  else:
      print('!!!Unexpected value in food type') # if you wrote something other than pizza or pasta, then it will print this message

print('Lets go have some ' + type_of_food + ' and '+ beverage + '!')

# Global Variables - One Change in One Place

Now that you have an idea of what using *input()* with *if else* statements can do, let's create our set our global variables. Think of this as "One Change in One Place". This cell will set the values for a many of the variables we'll use when trainig the model, saving outputs, and generating predictions. It can be useful to have "my_" at the start of all the variables that are defined in this chunk. That way if you're reading code somewhere else and see a variable that starts with "my_", you know exactly where it was initially defined. You can use something other than "my_", whatever you find useful. I have seen other people write their global variables in all caps.     

As you go through this exercise, you can think of other arguments you might have wanted to include in this section (e.g., *taget_size* or the *join_count* cutoff we used to prepare the metadata above). You may also find that you want fewer inputs here - lots of inputs can make it hard to follow the code. Maybe you are only interested in using the Xception architechture. There is a balance between elegance and readability. When developing code, we usually:
    

1.   Write code for one use case (e.g., continuous UFP trained on satellite images using Xception architecture)
2.   Think about the other use cases
3.   Write out the if else for all the changes that would need to be made
4.   Create One in Change One Place code cell and replace values with what are now your global variables



Another advantage of this "One Change in One Place" appoach is that you can easily turn it into loop(s) to train multiple models. What if you want to develop UFP and BC models trained on satellite and street-view images (i.e., four models)? You could set up loops to train them all one after the other.

The choices you'll make in the code cell below:
1. Type of images, satellite or street.
2. Pollutant you will model, UFP or BC.
3. Numeric form of pollutant, continuous or quintiles. If quintile, it can be numeric or categorical.
4. How long to train, put 10 epochs in the interest of time.
5. What model architecture you'll use, Xception, ResNet50V2 or DensNet121.


In [ ]:
########################### ONE CHANGE ONE PLACE #################################
# this code will affect the rest of the code. it asks for inputs from you and sets the names of the parameters for the model
# I should add more comments here explaining each one

# this opens the prompt below asking for user input. once you hit return, the value will be assinged to my_type_of_images_long
my_type_of_images_long = input("Train on which kind of images: satellite or street? ")

# this detects 'sat' in the input, that way we can just write 'sat' or 'satellite' or even misspell anything other than the first three letters
if 'sat' in my_type_of_images_long:  # if the input contains 'sat' then:
  my_type_of_images = 'sat'  # we will use 'my_type_of_images' when naming our output files, this gives a consistent name to our outputs regardless of how we spelled the input
  metadata['train_file'] = metadata['sat_file']  # create a column that identifies what images we will train the models on, this shows names and type
  my_vert_flip = True   # during training, we want vertical flipping for satellite images, but not for street images
else:
  if 'str' in my_type_of_images_long: # if the input contains 'str' then:
    my_type_of_images = 'str'  # we will include 'str' in the names output files
    metadata['train_file'] = metadata['str_file']  # create a column for training, but in this case it is filled with the values of the street view file names
    my_vert_flip = False  # during training, we do not want vertical flipping for street images
  else:
      print('!!!TYPO in train image type')  # if the input does not contain 'sat' or 'str', then it will print this message

my_ufp_or_bc = input("What pollutant: ufp or bc? ") # this asks for user input and sets the pollutant to either UFP of BC
my_pred_col = my_ufp_or_bc + '_pred'  # when we generate predictions, we will create a new colum named either ufp_pred or bc_pred

# do we want to train on the continous values or the quintiles?
my_cont_or_quint_long = input("What form of the pollutant: continuous or quintiles? ")

# like with type of images, this will detect the first couple of letters, which lets us have some typos
if 'cont' in my_cont_or_quint_long:
  my_cont_or_quint = 'continuous' # like with type of images, this sets the variable that we will use in other spots
  my_y_col = my_ufp_or_bc  # this set the y column for training and predictions
  my_target_class_mode = 'raw'  # this tells the model that the y column is continous (i.e. treat as it is, 'raw')
  my_activation = 'linear'  # the activation function is linear for continuous y, bascially a linear regression
  my_output_units = 1  # we want a single value as the output, that is the form of output from linear activation
  my_loss = rmse  # the loss function in the model, root mean squared error for continous outcomes. we difeined this at the start. we could also use 'mae' which is mean absolute error
  my_loss_metric = rmse  # the metric to track during training. this will be tracked in the training set and in the validation set
  my_metric_of_interest = 'val_rmse'  # the callbacks will monitor root mean squared error in the validation set, more on this below
  my_max_or_min_metric = 'min'  # the goal is to minimize rmse
  my_type_of_y = 'numeric' # this is just to help with the printed output at the end of this chunk

else:
  if 'quint' in my_cont_or_quint_long:  # instead of training on continous values, we could train on the quintiles
    my_cont_or_quint = 'quintiles'   # this sets the variable that we can use in other places
    my_type_of_quint_y_long = input("What form of quintile: numeric or categorical? ") # even when using quintiles, we could treat them as continuous numbers from 1 - 5 or as seperate categories
    if 'cat' in my_type_of_quint_y_long: # if we want to treat the quintiles as catigorical
      my_type_of_y = 'categorical'
      my_y_col = my_ufp_or_bc + '_quint_cat' # this set the y column for training and predictions to either ufp_quint_cat or bc_quint_cat, recall that we created this column as strings of 1 - 5
      my_target_class_mode = 'categorical' # this tells the model that the y column is categorical
      my_activation = 'softmax'  # categorical uses a softmax activation function
      my_output_units = 5  # now that we are looking at 5 categories, we want an output that is the probability of each category
      my_loss = 'categorical_crossentropy' # the loss function in the model, categorical_crossentropy for categorical outcomes.
      my_loss_metric = 'accuracy' # the metric to track during training. this will be tracked in the training set and in the validation set. for categorical, we are interested in accuracy
      my_metric_of_interest = 'val_accuracy'  # the callbacks will monitor accuracy in the validation set, more on this below
      my_max_or_min_metric = 'max' # we want maximum accuracy
    else:
      if 'num' in my_type_of_quint_y_long:
        my_type_of_y = 'numeric'  # we could chose to train on quitiles, but treat them as continous numbers from 1 - 5
        my_y_col = my_ufp_or_bc + '_quint_cont'  # this set the y column for training and predictions, recall that this was the original quintile column of numbers from 1 - 5
        my_target_class_mode = 'raw'  # because we are treating them as continuous, the rest of the parameters are for linear
        my_activation = 'linear' # activation , ouptus, and loss are all the same as for continuous
        my_output_units = 1
        my_loss = rmse
        my_loss_metric = rmse
        my_metric_of_interest = 'val_rmse'
        my_max_or_min_metric = 'min'
      else:
        print('!!!TYPO in Form of quintile')
  else:
    print('!!!TYPO in continuous vs quintile')

# how long do we want to train the model for? typically we will train up to 100 epochs, but in the interest of time, set to 10
my_number_of_epochs = input("Train for how many epochs? 10 epochs takes about 25 mins ")

# which model architecture do you want to use?
my_model_architecture_long = input("What model architecture do you want to use? xception, resnet50V2, or densenet121? ")

# this allows us to make some typos in model architecture input. If you get the first three letters correct, then it'll set model architecture name in a predictable way
if 'xce' in my_model_architecture_long:
  my_model_architecture = 'xcep'
  my_pre_pro_fn = K.applications.xception.preprocess_input # the different architectures have different preprocessing functions
  my_batch_size = 32  # batch size is the number of observations the model will use for gradient descent
else:
  if 'resn' in my_model_architecture_long:
    my_model_architecture = 'resn'
    my_pre_pro_fn = K.applications.resnet_v2.preprocess_input
    my_batch_size = 32
  else:
    if 'dens' in my_model_architecture_long:
      my_model_architecture = 'dens'
      my_pre_pro_fn = K.applications.densenet.preprocess_input
      my_batch_size = 32  # This model is a bit more comuationally intensive and required a smaller batch size
    else:
      print('!!!TYPO in model architecture ')

# this gives a printout that is easy to read. it is handy for:
  # 1. checking to make sure the inputs are what you want
  # 2. when you come back after hours of training and ask yourself: "what did I just train?"
print('You are using ' + my_model_architecture_long + ' trained on ' + my_type_of_images + ' images to predict the ' + my_y_col + ' column (' + my_ufp_or_bc +' '+ my_cont_or_quint +' as '+ my_type_of_y + ')')


With your global variables set, you can now set up image pre-processing exactly how you want it without having to modify the code in the next cell.

In [ ]:
generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function= my_pre_pro_fn, # this is the preprocessing function defined above
                                                      horizontal_flip=True, # we flip the images for more robust training
                                                      vertical_flip = my_vert_flip) # set to false if using street images

# specify the dataframe for training. we just need the training set with the target column (i.e., the 'y', the pollutant we want to train on) and the filepaths of the training imagaes
train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='train', [my_y_col, 'train_file']].reset_index(drop=True),
                                                directory= image_dir,  # we already defined our image directory when we were ploting the pictures
                                                x_col= 'train_file',  # recall that this was set to either str_file or sat_file
                                                y_col= my_y_col,  # this is either ufp, ufp_cat (1 - 5 continuous), ufp_cat_str (1 - 5 categorical), or the bc equivalent
                                                class_mode= my_target_class_mode,    # is the outcome continuous ('raw') or categorical?
                                                target_size=(240, 240),
                                                color_mode='rgb',
                                                batch_size= my_batch_size,
                                                shuffle=True)

# the validate is the same except with the validation set
validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='validate', [my_y_col, 'train_file']].reset_index(drop=True),
                                                     directory= image_dir,
                                                     x_col= 'train_file',
                                                     y_col= my_y_col,
                                                     class_mode= my_target_class_mode,
                                                     target_size=(240, 240),
                                                     color_mode='rgb',
                                                     batch_size= my_batch_size,
                                                     shuffle=False)

You will see output telling you how many validated image filenames were found. Recall that we had 3362 sites, with two images each that makes 6724 images. That is reassuring! If any images were missing, it will tell you here.   

***Try later:*** *You can start a new runtime and try training on satellite images without moving the "missing_images_19" images. How does that impact the output when you run the code cell above?*


# Define Callbacks

"Callbacks" are actions that we want Keras to do during trainig and we can set conditions for those actions based on model performance in the validation or training sets. In our case we want to define criteria for early stopping and reducing learning rates. We also want to save a log of model performance and save only the best model. We could save the model from each epoch, but that can take up a lot of space and we probably aren't interested in anything other than the best model. If you're tight on space, you can always just save the models wieghts instead of the entire model.

In [ ]:
# define callbacks

# stop the training if the metric of interest (RMSE or accuracy) doesn't improve during a certain number of epochs (set to 15 here). there's no sense in continuous training if the model isn't leraning
early_stopping = K.callbacks.EarlyStopping(monitor= my_metric_of_interest, patience=15, mode= my_max_or_min_metric)

# reduce learning by a certain factor if the metric of interest (RMSE or accuracy) doesn't improve during a certain number of epochs (set to 5 here)
reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor= my_metric_of_interest, factor=0.1, patience=5, mode= my_max_or_min_metric, verbose=1) # verbose = 1 gives us output every time the learning rate changes

# save a log of model performance. notice how the file name will change depending on which model we are training (pollutant_type-of-image_architecture_max-epochs)
csv_logger = K.callbacks.CSVLogger('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_log/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture +'_model_logger_'+ my_number_of_epochs+'e_lr10e-3.csv')

# save the best model
model_checkpoint = K.callbacks.ModelCheckpoint('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_best_model_'+ my_number_of_epochs+'e_lr10e-3.hdf5', monitor= my_metric_of_interest, mode= my_max_or_min_metric, save_weights_only=False, save_best_only=True)

# this will print out so you can see what the model name will be
'/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_best_model_'+ my_number_of_epochs+'e_lr10e-3.hdf5'


# Compile and Train Model

Next we define the model compile function. Compiling sets the shape of input tensors, the model architecture, any initial weights, the activation function, and several other model parameters you can see in the code below.

If we are training on a continous y, then we use a linear activation function and RMSE as the loss function. If we are training on a categorical y, then we use a softmax activation function with accuracy as the loss function. Now you can see how setting all those global variables in one place is paying off!

**Initial Weights**

We'll use inital weights from a model that has already been trained. This is called "transfer learning". Here we'll use weights from models trained on the "ImageNet" database. It has millions of labeled images and is used in research. The images have nothing to do with air pollution, but it costs us nothing to use these initial weights so we may as well. If we had some other trained models, we could try using their initial weights as well.  

***Try later:*** *You could try training without using imagenet initial weights or with the initial weigths from some other models. How does that impact model performance? Do you need to train longer in order to get good perfromance?*

Note on "include_top":
https://stackoverflow.com/questions/46036522/defining-model-in-keras-include-top-true


In [ ]:
def get_compiled_model():
    # define model input. Xception and ResNet50 can take various input shapes, but EfficientNetB2 requires 260 x 260, 3 (the 3 is the color channels)
    model_input = K.layers.Input(shape=(240, 240, 3), dtype='float32', name='input')

    # define base, one of three models. There are many others to pick from. We are using imagenet initial weights for all of them and doing so requires include_top = False
    if my_model_architecture == 'xcep':
      conv_base = K.applications.Xception(include_top=False, weights= "imagenet", input_tensor=model_input)  # include_top = False in order to use initial weights (i.e. transfer learning)
    else:
      if my_model_architecture == 'resn':
        conv_base = K.applications.ResNet50V2(include_top=False, weights= "imagenet", input_tensor=model_input)
      else:
        if my_model_architecture == 'dens':
          conv_base = K.applications.DenseNet121(include_top=False, weights= "imagenet", input_tensor=model_input)

    model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
    model_output = K.layers.Dense(units= my_output_units, activation= my_activation)(model_output)  # this is what makes it a regression model, for categorical: K.layers.Dense(units=10, activation='softmax')(model_output), class_mode in generators would also need to be changed to 'categorical'
    model = K.models.Model(inputs=model_input, outputs=model_output)
    model.compile(
        optimizer=K.optimizers.Nadam(learning_rate = 0.01),  # may want to tune this learning rate. seems like 0.01 and 0.001 are alright. 0.0001 is too slow
        loss = my_loss,   # we defined rmse in the first code chunk
        metrics = my_loss_metric  # you can track multiple metrics if you want, e.g., for continuous you could do [rmse, 'mae']
    )
    return model

In [ ]:
# # we've included code showing what it would look like if you filled in all the values yourself instead of using variables defined
# # this example is for an Xception model architechture with output of categorcial (i.e. classifier) quintiles
#
#   # define compiling model
#   def get_compiled_model():
#       # define model
#       model_input = K.layers.Input(shape=(240, 240, 3), dtype='float32', name='input')
#       conv_base = K.applications.Xception(include_top=False, weights= "imagenet", input_tensor=model_input)
#       model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
#       model_output = K.layers.Dense(units= 5, activation= 'softmax')(model_output)  # this is what makes it a regression model, for categorical: K.layers.Dense(units=10, activation='softmax')(model_output), class_mode in generators would also need to be changed to 'categorical'
#       model = K.models.Model(inputs=model_input, outputs=model_output)
#       model.compile(
#           optimizer=K.optimizers.Nadam(learning_rate = 0.01),  # may want to tune this learning rate. seems like 0.01 and 0.001 are alright. 0.0001 is too slow
#           loss = 'categorical_crossentropy',
#           metrics = ['accuracy']
#       )
#       return model


You have defined the get_compiled_model() function, now we want to actually compile it. We could have just run each line separetely, but we find it nice to have them all in once spot.

> ***Compiling the model will take 2-10 seconds. You should see a progess bar.***

In [ ]:
model = get_compiled_model()

Now you are ready to train (i.e., fit) your model! If you set epochs to 10, this should take 20-30 mins. You will see progress bars for each epoch.

> ***Training will take a while (~25 mins for 10 epochs) , so plan accordingly (i.e. go grab a coffee if you want!).***


In [ ]:
model.fit(train_generator,
          validation_data=validate_generator,
          epochs= int(my_number_of_epochs), # we have set it to only 10 epochs in the interest of time, this should take about 25  mins
          steps_per_epoch=int(np.ceil(train_generator.samples/train_generator.batch_size)),
          validation_steps=int(np.ceil(validate_generator.samples/validate_generator.batch_size)),
          callbacks=[early_stopping, reduce_lr_on_plateau, csv_logger, model_checkpoint]) # all the callbacks we defined earlier

You have just trained your model using the training and validation sets. The log file that was saved will tell you how the model performed in the training and validation sets, but those values will be biased. You should evaluate the model using data that the model has not seen, you should use the test set. Start by generating predictions in the test set.             

# Generate Predictions

As we did for the train and validation data sets, we will create the test generator. There is no *y_col* required for the test generator because it will simply be using the *x_col* (i.e., images) to generate predictions. Notice how the dataframe is using all the data, not just the test set data. We find it handy to generate predictions for all the data and then filter to just the test set when evaluating the model.

In [ ]:

generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function= my_pre_pro_fn,
                                                      horizontal_flip=False,
                                                      vertical_flip = False) # you do not want to flip the images when generating predictions

# you could just generate predictions for the test set, but here we will generate predictions for the entire data set
# keep this in mind for when we later evaluate the results. at that point we will have to make sure we are only evaluating in the test set.
test_generator = generator.flow_from_dataframe(dataframe=metadata[[my_y_col, 'train_file']].reset_index(drop=True),
                                               directory= image_dir,
                                               x_col= 'train_file',
                                               class_mode= 'input', # or None?
                                               target_size=(240, 240),
                                               color_mode='rgb',
                                               batch_size= my_batch_size,
                                               shuffle=False)



Recall that we specified in the callbacks to save only the best model, so now we will load that model in order to generate predictions. We need to put the predictions somewhere, so we will also prepare a dataframe based on the metadata. The predictions will be added as a column to that dataframe. Useful information to have in this "results" dataframe are:  

- *long* and *lat* (in order to plot)
- *train_file* (if we have strange predictions, we may want to inspect the image)   
- observed ufp, bc, and quintile (for comparing to predictions)
- temp, relhum, winspeed (for temporal adjustments)
- prediction (this is it what it is all about!)

When generating predictions, Keras will skip any rows that don't have valid images. This means that if there are missing images you will have fewer predictions than there are rows in the results datafram. When setting up the results dataframe, it is good to verify the existence of all the image files references in the dataframe and to remove any rows that refer to invalid/missing images.  

Finally, we'll save the dataframe as a csv file.

In [ ]:
# load the model you just trained, this will take about 30s
model = K.models.load_model('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_best_model_'+ my_number_of_epochs+'e_lr10e-3.hdf5', custom_objects={'rmse': rmse})


> ***Generating predictions in the next code cell can take 1-2 minutes to complete, you'll get a progress bar***




In [ ]:

# prepare a new dataframe based on the columns of interest in the metadata
results = metadata[['id', 'long', 'lat', 'train_file', 'set', 'ufp', 'bc', 'ufp_quint_cont', 'bc_quint_cont', 'temp_f', 'relhum_pct', 'windspeed_kts']]

# check to see if each image file exists and remove rows from the results dataframe for which image files don't exist.
results['file_exists'] = results.apply(lambda row: os.path.isfile(image_dir + row.train_file), axis = 1)
results = results.loc[results['file_exists'] == True]

# if categorical, then the prediction gives a prediction for probability of each category. take the max
# if continuous, then we just take the prediction.
if 'cat' in my_y_col:
  results[my_pred_col] = model.predict(x=test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size))).argmax(axis = -1)
else:
  results[my_pred_col] = model.predict(x=test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size)))

# save the results as a csv file
results.to_csv(path_or_buf='/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_predictions/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_model_predictions_'+ my_number_of_epochs+'e_lr10e-3.csv', index=False)
results

Congrats! You have trained a deep CNN and used it to generate air pollution predictions using images. What now?

Now you should evaluate your model.

Also, if you trained categorical quintiles, take a look at the observed vs the predicted quintiles. What do you notice? What are the ranges? Python uses "zero-based indexing", which means counting starts at 0, whereas in R counting starts at 1. Some people have strong opinions about this, but there's no "right" way, it's just important to keep in mind when swithcing between R and Python.




**Pre-Trained Models**

In this workshop we only had enough time for you to train for 10 epoch. Typically we would try training much longer (e.g., up to 100). We have provided models that we trained for 30 epochs that you can also use to generate a predictions file. You can run the code in the cells below to move the pre-trained models into your Google Drive and to get an overview of what the models they are. The code will also move the pre-trained model logs.  

> ***Copying the files will take about 60 seconds***




In [ ]:
# this cell will move the pre-trained models into your Google Drive folder

# list the pre-trained models
pre_trained_models = os.listdir('/content/drive/MyDrive/sharp_ml_course/pre_trained_models/ufp_colombia_pre_trained_model')

# set the origin and destination directories
from_dir = '/content/drive/MyDrive/sharp_ml_course/pre_trained_models/ufp_colombia_pre_trained_model/'
to_dir = '/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/'

# copy to the destination directory
for i in pre_trained_models:
  # renames the files, instead of changing the file's name, we change the directory
  shutil.copyfile(from_dir + i, to_dir + i)

# repeat for the model logs
pre_trained_model_logs = os.listdir('/content/drive/MyDrive/sharp_ml_course/pre_trained_models/ufp_colombia_pre_trained_model_logs')
from_dir = '/content/drive/MyDrive/sharp_ml_course/pre_trained_models/ufp_colombia_pre_trained_model_logs/'
to_dir = '/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_log/'
for i in pre_trained_model_logs:
  shutil.copyfile(from_dir + i, to_dir + i)

# print out names of the pre-trained models
pre_trained_models

In [ ]:
# this code extracts information from the model name in order to help us understand the models
# this shows you how long names can seem clunky, but can be very useful. everything we need to know about the model is in the model's name
# this code cell will also display some of the important model info in a table
import re
d = []
for x in pre_trained_models:
  d.append((x, re.findall('ufp_quint_cat|ufp_quint_cont|ufp', x)[0], re.findall('sat|str', x)[0], re.findall('xcep|resn|dens', x)[0], re.findall('model_(.*)e_lr', x)[0]))
pre_trained_model_df = pd.DataFrame(d, columns = ('model_name', 'outcome', 'type_of_image', 'model_arch', 'number_epochs'))
display(pre_trained_model_df)

The cell below will load one of the models in your *sharp_d2_UFP/output/model/* directory. That directory now contains the model you trained as well as the pre-trained models that we have provided. All those models are displayed in the table above. You can identify your model because *number_epochs* = 10, whereas *number_epochs* = 30 for all the pre-trained models.  

**You don't need to run the code cell below** if you want to proceed with model evaluation using your model. You can run the code cell below ONLY if you want to load one of the pre-trained models to generate predictions for model evalutation (i.e. instead of using the model you trained).  

> ***Loading model and generating predictions in the next code cell can take several minutes to complete. You will be prompted with an input below the code cell. Make sure to enter the row index to indicate which model you want to use when prompted.***

In [ ]:
#####################################################################################################
########### code for loading a pre-trained model and using it to generate predictions ###############
#####################################################################################################

#####################################################################################################
######### DO NOT RUN THIS CODE IF YOU WANT TO USE THE MODEL YOU JUST TRAINED ########################
#####################################################################################################
selected_model = int(input('Using row index (i.e., number), indicate which model you want to use: '))
print('You selected the ' + pre_trained_model_df.loc[selected_model, 'model_name'] + ' pre-trained model')

# this loads the model
model = K.models.load_model('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/' + pre_trained_model_df.loc[selected_model, 'model_name'], custom_objects={'rmse': rmse})

# this sets the values
my_y_col, my_pred_col, my_type_of_images, my_model_architecture, my_number_of_epochs = pre_trained_model_df.loc[selected_model, 'outcome'], 'ufp_pred', pre_trained_model_df.loc[selected_model, 'type_of_image'], pre_trained_model_df.loc[selected_model, 'model_arch'], pre_trained_model_df.loc[selected_model, 'number_epochs']

if my_model_architecture == 'xcep':
  my_pre_pro_fn = K.applications.xception.preprocess_input
else:
  if my_model_architecture == 'resn':
    my_pre_pro_fn = K.applications.resnet_v2.preprocess_input
  else:
    if my_model_architecture == 'dens':
      my_pre_pro_fn = K.applications.densenet.preprocess_input


# if the runtime has been disconnected/terminated the metadata will need to be read in again. as a shortcut, we have the processed metadata here
metadata = pd.read_csv("/content/drive/MyDrive/sharp_ml_course/pre_trained_models/ufp_colombia_metadata.csv")
if my_type_of_images == 'sat':
  metadata['train_file'] = metadata['sat_file']
else:
  metadata['train_file'] = metadata['str_file']

my_batch_size = 32
image_dir = '/content/images/'

# need to redefine the test generator
generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function= my_pre_pro_fn,
                                                      horizontal_flip=False,
                                                      vertical_flip = False)
test_generator = generator.flow_from_dataframe(dataframe=metadata[[my_y_col, 'train_file']].reset_index(drop=True),
                                               directory= image_dir,
                                               x_col= 'train_file',
                                               class_mode= 'input',
                                               target_size=(240, 240),
                                               color_mode='rgb',
                                               batch_size= my_batch_size,
                                               shuffle=False)

# prepare a new dataframe based on the columns of interest in the metadata
results = metadata[['id', 'long', 'lat', 'train_file', 'set', 'ufp', 'bc', 'ufp_quint_cont', 'bc_quint_cont', 'temp_f', 'relhum_pct', 'windspeed_kts']]

# check to see if each image file exists and remove rows from the results dataframe for which image files don't exist.
results['file_exists'] = results.apply(lambda row: os.path.isfile(image_dir + row.train_file), axis = 1)
results = results.loc[results['file_exists'] == True]

# if categorical, then the prediction gives a prediction for probability of each category. take the max
# if continuous, then we just take the prediction.
if 'cat' in my_y_col:
  results[my_pred_col] = model.predict(x=test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size))).argmax(axis = -1)
else:
  results[my_pred_col] = model.predict(x=test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size)))

# save the results as a csv file
results.to_csv(path_or_buf='/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_predictions/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_model_predictions_'+ my_number_of_epochs+'e_lr10e-3.csv', index=False)
results

# Model Evaluation

You may be coming back to a session that was disconnected/terminated and may need to re-define the global variables. You can do that up at the "One Change One Place" section or you can do it right here.  

***You don't need to run the two code cells below if your session was not disconnected/terminated***


In [ ]:
# this code extracts information from the model names in order to help undestand what the model was trained on
# get a list the available models, this will be a mix of pre-trained models and models you have trained today
available_models = os.listdir('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/')
d = []
for x in available_models:
  d.append((x, re.findall('ufp_quint_cat|ufp_quint_cont|ufp', x)[0], re.findall('sat|str', x)[0], re.findall('xcep|resn|dens', x)[0], re.findall('model_(.*)e_lr', x)[0]))
available_models_df = pd.DataFrame(d, columns = ('model_name', 'outcome', 'type_of_image', 'model_arch', 'number_epochs'))
display(available_models_df)


In [ ]:
#########################################################################################################
############### No need to run this cell if your session was not disconnected/terminated  ###############
#########################################################################################################

print('The model that you trained should have 10 for number_epochs')
selected_model = int(input('Which model results do you want to use for model evaluation? Inidcate the row index, which starts at 0 (i.e., number) '))
print('You selected the ' + available_models_df.loc[selected_model, 'model_name'] + ' model')

# this sets the values
my_y_col, my_pred_col, my_type_of_images, my_model_architecture, my_number_of_epochs = available_models_df.loc[selected_model, 'outcome'], 'ufp_pred', available_models_df.loc[selected_model, 'type_of_image'], available_models_df.loc[selected_model, 'model_arch'], available_models_df.loc[selected_model, 'number_epochs']


**Model Log**

Before we dig into predictions, we can take a look at the model log to see the change in performance in the training and validation sets during training. Perfromance in the training set is usally higher that in the validation set, though we hope that they end up being pretty close. If they are not close, then that is an indication that the model is overfit. You would also like to see the lines in the plot flattening out in later epochs. If the lines are still very wiggly, then the models could probably benefit from more training epochs.  

If you are training on categorical quintiles, the y axis will be accuracy. Randomly guessing quintiles would results in an accuracy of 0.2,

In [ ]:
# read in the model log
train_log = pd.read_csv('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_log/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_model_logger_'+ my_number_of_epochs+'e_lr10e-3.csv')

print('Looking at ' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_model_logger_'+ my_number_of_epochs+'e_lr10e-3')

# if we are training on categorical, then we want to look at accuracy.
if 'cat' in my_y_col:
  plt.plot(train_log['epoch'], train_log['val_accuracy'])
  plt.plot(train_log['epoch'], train_log['accuracy'])
  plt.legend(["validation", "train"], loc="upper left")
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.ylim(ymin = 0, ymax = 1)
else:  # if we are training on continuous or numeric quintile, then we want to look at RMSE.
  plt.plot(train_log['epoch'], train_log['val_rmse'])
  plt.plot(train_log['epoch'], train_log['rmse'])
  plt.legend(["validation", "train"], loc="upper left")
  plt.ylabel('RMSE')
  plt.xlabel('Epoch')
  plt.ylim(ymin = 0, ymax = 100000) # might need to set a limit to make it interpretable, sometimes the RMSE can start very very high



**Comparing Observed to Predicted**

Read in the results dataframe from the csv you saved.

Recall that we trained on two images at each road segment (either zoom 18 and 19 or angles 1 and 2) and then generated predictions on two images at each road segment. You should take the average of the two predictions as the predicted value for a road segment.

In [ ]:
# read in the predictions
results = pd.read_csv('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model_predictions/' + my_y_col + '_' + my_type_of_images + '_' + my_model_architecture + '_model_predictions_'+ my_number_of_epochs+'e_lr10e-3.csv')

# group them by site id and take the mean prediction
results_gr = results.groupby(['id', 'long', 'lat', 'set', 'ufp', 'bc', 'temp_f', 'relhum_pct', 'windspeed_kts'])[my_pred_col].mean().reset_index()

# create a dataframe that is just the test set.
results_gr_test = results_gr.loc[results_gr['set'] == 'test', ]
results_gr_val = results_gr.loc[results_gr['set'] == 'validate', ]
results_gr_train = results_gr.loc[results_gr['set'] == 'train', ]

We can compare observed to predicted in a scatter plot. On the left is all the data and on the right is just in the test set. Model evaluation should only be done with the independent test set, but it can be interesting to look at all the data as well.  

In [ ]:
if 'ufp' in my_y_col:
  obs_col = 'ufp'
else:
  obs_col = 'bc'

f = plt.figure(figsize=(12,5))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.plot(results_gr[my_pred_col], results_gr[obs_col], 'bo')
ax2.plot(results_gr_test[my_pred_col], results_gr_test[obs_col], 'bo')

ax.set_xlabel('Predicted ' + obs_col + ' Quintile') # upadate label to quint vs continous
ax2.set_xlabel('Predicted ' + obs_col + ' Quintile')
ax.set_ylabel('Observed ' + obs_col + ' Concentration')

ax.set_title('Predicting ' + my_y_col + ' in All Data (' + my_model_architecture + ')')
ax2.set_title('Predicting ' + my_y_col + ' in Test Set (' + my_model_architecture + ')')

It's hard to pass or fail the eyeball test, so lets compare the predictions to the observed values in a regression. That allows us to state what proportion of the observed spatial variation in UFP concentrations in the test set are explained by the model. Here is where we include meterological conditions (i.e., weather) in order to adjust for weather-related temporal variations in the observed concentrations during monitoring.




In [ ]:
# first regress the predictions and meterological conditions onto observed values in the test set
multi_x_val, y_val = results_gr_val[[my_pred_col, 'temp_f', 'relhum_pct', 'windspeed_kts']], results_gr_val[obs_col] # .array.reshape(-1,1) for the x is there is only a single predictor

# this gives us our temporal adjustment model in the validation set
results_linear_model_val = LinearRegression().fit(multi_x_val, y_val)

print(f"Temporal adjustment slopes in validation set are: {results_linear_model_val.coef_}")


In [ ]:
# then use the temporal adjusment model on the test set data
multi_x_test, y_test = results_gr_test[[my_pred_col, 'temp_f', 'relhum_pct', 'windspeed_kts']], results_gr_test[obs_col] # .array.reshape(-1,1) for the x is there is only a single predictor

results_gr_test = results_gr_test.assign(temp_adj_pred = results_linear_model_val.predict(multi_x_test))

x_test = results_gr_test['temp_adj_pred'].array.reshape(-1,1)

results_linear_model_test = LinearRegression().fit(x_test, y_test)
r_sq = results_linear_model_test.score(x_test, y_test)

y_test_pred = results_linear_model_test.predict(x_test)
mse = mean_squared_error(y_test, y_test_pred)

print(f"MSE in test set is: {mse}")
print(f"R2 in test set is: {r_sq}")
print(f"Test set regression slopes is: {results_linear_model_test.coef_}")
print(f"Test set intercept is: {results_linear_model_test.intercept_}")


You can record the MSE, R2, and your model details here:
https://docs.google.com/spreadsheets/d/18SN3iT4L6ixL7FUVpRNr3khos51TUM4GcS-4T-zvu-c/edit?usp=sharing

If we wanted to cheat, we could do the temporal adjustment using all the data and evaluate the predictions against all the observed values (i.e., not use an independent test set). Does the R2 look better?

In [ ]:
x, multi_x, y = results_gr[my_pred_col].array.reshape(-1,1), results_gr[[my_pred_col, 'temp_f', 'relhum_pct', 'windspeed_kts']], results_gr[obs_col] # .array.reshape(-1,1) for the x is there is only a single predictor

results_linear_model = LinearRegression().fit(multi_x, y)

r_sq = results_linear_model.score(multi_x, y)

print(f"R2 in all data is: {r_sq}")

In [ ]:
# now use the temporal adjustment model to generate predictions in all the data.
# This is so we can visualize the prediction. It isn't to evaluate model performance.
results_gr = results_gr.assign(temp_adj_pred = results_linear_model_val.predict(multi_x))
results_gr

# Plot results




In [ ]:
# we can see how using a linear regression temporal adjustment of categorical values results in continuous values
plt.plot(results_gr['temp_adj_pred'], results_gr[obs_col], 'bo')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey=True, figsize=(20, 10))
results_gr['ufp_pred_level'] = pd.qcut(results_gr['temp_adj_pred'], 9)
sns.scatterplot(x='long', y='lat', hue= 'ufp_pred_level', palette = sns.color_palette("RdYlGn_r", 9), data= results_gr, s = 8, ax = ax1).set(title='Predicted UFP')

results_gr['ufp_level'] = pd.qcut(results_gr['ufp'], 9)
sns.scatterplot(x='long', y='lat', hue= 'ufp_level', palette = sns.color_palette("RdYlGn_r", 9), data= results_gr, s = 8, ax = ax2).set(title='Observed UFP')

results_gr['residuals'] = results_gr['ufp'] - results_gr['temp_adj_pred']

res_mean = np.mean(results_gr['residuals'])
res_std = np.std(results_gr['residuals'])

results_gr['residuals_norm'] = pd.qcut((results_gr['residuals'] - res_mean)/res_std, 9)

sns.scatterplot(x='long', y='lat', data= results_gr, hue= 'residuals_norm', palette = sns.color_palette("vlag", 9), s = 8, ax = ax3).set(title='Normalized Residuals')

# Class Activation Maps

What part of the images are important? We can't say for sure, but we can look at how the model responds to individual images. These are "Class Activation Maps" (CAMs). They show what part of the image is contributing to the class prediction. It is important to note that this is for "class" predictions, i.e., categorical predictions. We can only make CAMs for models that generate categorical predictions. We cannot make CAMs for models that generate continous predictions.  

The code below is made to run on its own without any of the inputs above. This lets you come back to this file to explore images without having to worry about any of the code above. You also have the option to use one of the pre-trained models that you moved into your Google Drive using the code just before the "Model Evaluation" section.   

In [ ]:
# get a list the available models, this will be a mix of pre-trained models and models you have trained today
available_models = os.listdir('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/')

# extract information from the model names in order to help undestand what the model was trained on
# it's imporant to note the "outcome" column, you can only select a model that predicts "cat" for CAMs
d = []
for x in available_models:
  d.append((x, re.findall('ufp_quint_cat|ufp_quint_cont|ufp', x)[0], re.findall('sat|str', x)[0], re.findall('xcep|resn|dens', x)[0], re.findall('model_(.*)e_lr', x)[0]))
available_models_df = pd.DataFrame(d, columns = ('model_name', 'outcome', 'type_of_image', 'model_arch', 'number_epochs'))
display(available_models_df)


NameError: ignored

> ***Loading a model in the next code cell takes about 15 seconds. Don't forget to enter the row number when prompted.***

In [ ]:
# run this code to pick and load a model
selected_model = int(input('Which model do you want to use for Class Activation Maps? Inidcate the row index (i.e, number) '))
print('You selected the ' + available_models_df.loc[selected_model, 'model_name'] + ' model')
if bool(re.search('cat', available_models_df.loc[selected_model, 'outcome'])) == False:
  raise SystemError('Cannot use continuous outcome model for CAMs. Re-run this cell and select a model that predicts categorical outcomes!')

# this loads the model
model = K.models.load_model('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/' + available_models_df.loc[selected_model, 'model_name'], custom_objects={'rmse': rmse})

# this sets the values
my_y_col, my_pred_col, my_type_of_images, my_model_architecture, my_number_of_epochs = available_models_df.loc[selected_model, 'outcome'], 'ufp_pred', available_models_df.loc[selected_model, 'type_of_image'], available_models_df.loc[selected_model, 'model_arch'], available_models_df.loc[selected_model, 'number_epochs']


In [ ]:

# print out a summary, you want the name of the last layer before the activation function
print(model.summary())

Some useful exaples of what we will do:  
https://medium.com/@stepanulyanin/implementing-grad-cam-in-pytorch-ea0937c31e82
  
 https://www.kaggle.com/code/lov4jin/densenet-imagenet-pretrained-vs-retrained-gradcam

In [ ]:
# define model builder, pre-processing and decoding
# this is specific to the model architecture
if 'xce' in my_model_architecture:
  cam_model_builder = K.applications.xception.Xception
  cam_pre_pro_fn = K.applications.xception.preprocess_input # the different architectures have different preprocessing functions
  cam_decode_fn = K.applications.xception.decode_predictions
  last_conv_layer_name = 'block14_sepconv2_act'  # you can see where this layer is in the model.summary() output
else:
  if 'res' in my_model_architecture:
    cam_model_builder = K.applications.resnet_v2.ResNet50V2
    cam_pre_pro_fn = K.applications.resnet_v2.preprocess_input
    cam_decode_fn = K.applications.resnet_v2.decode_predictions
    last_conv_layer_name = 'block14_sepconv2_act' # you can see where this layer is in the model.summary() output
  else:
    if 'den' in my_model_architecture:
      cam_model_builder = K.applications.densenet.DenseNet121
      cam_pre_pro_fn = K.applications.densenet.preprocess_input
      cam_decode_fn = K.applications.densenet.decode_predictions
      last_conv_layer_name = 'block14_sepconv2_act' # you can see where this layer is in the model.summary() output

model_builder = cam_model_builder
img_size = (240, 240)
preprocess_input = cam_pre_pro_fn
decode_predictions = cam_decode_fn

# take a look at an image we will be working with
if my_type_of_images == 'str':
  first_image = 'angle1'
  second_image = 'angle2'
else:
  first_image = '18'
  second_image = '19'

# this will look at image number 100. Further down we have streamlined the code so you can easily pick different images.
img_path = '/content/images/images_' + first_image +'/00100.jpg'
display(Image(img_path))

In [ ]:
# read in an image and turn it into an array
def get_img_array(img_path, size):
    img = K.preprocessing.image.load_img(img_path, target_size=size)
    array = K.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

# take the array of the single image and give it to the model to generate a prediction
# instead having the prediction as the output, take a look at how the last layer responds
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# let's see how those last two function work
# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.title('Class Probs = ' + ''.join(str(x) for x in preds))
plt.show()

In [ ]:
# now we want to overlay that heatmap onto the image that was used to generate it
def save_and_display_gradcam(read_img_path, heatmap, cam_name = 'grad-cam-image.jpg', cam_path='/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/grad_cam_images/', alpha=0.6):
    # Load the original image
    img = K.preprocessing.image.load_img(read_img_path)
    img = K.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = K.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = K.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = K.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    cam_path_and_name = cam_path + cam_name
    superimposed_img.save(cam_path_and_name)

    # Display Grad CAM
    display(Image(cam_path_and_name)) # maybe take this out of the function in order to display multiple in one plot


In [ ]:
# now save and display the image. give the image a meaningful name. here we name the pollutant, the type of image, and the image number
save_and_display_gradcam(img_path, heatmap, cam_name = 'grad_cam_ufp_zoom_resn_image0001.jpg')

# Streamlined CAM Code

The code cell below has all the code to generate, save, and display a CAMs for each image at a given site. It is the same code from the cell above, but all in one cell to make it easier to use.

In [ ]:
image_number = input('Image number (max 5916): ')

image_number = f"{int(image_number):05}"

if my_type_of_images == 'str':
  first_image = 'angle1'
  second_image = 'angle2'
else:
  first_image = '18'
  second_image = '19'

first_img_path = '/content/images/images_' + first_image +'/'+ image_number +'.jpg'
second_img_path = '/content/images/images_' + second_image +'/'+ image_number +'.jpg'

first_img_array = preprocess_input(get_img_array(first_img_path, size=img_size))
model.layers[-1].activation = None
first_preds = model.predict(first_img_array)
first_heatmap = make_gradcam_heatmap(first_img_array, model, last_conv_layer_name)

second_img_array = preprocess_input(get_img_array(second_img_path, size=img_size))
model.layers[-1].activation = None
second_preds = model.predict(second_img_array)
second_heatmap = make_gradcam_heatmap(second_img_array, model, last_conv_layer_name)

save_and_display_gradcam(first_img_path, first_heatmap, cam_name = 'grad_cam_' + my_model_architecture +'_' + my_y_col + '_' + first_image + 'image_'+ image_number +'.jpg')
save_and_display_gradcam(second_img_path, second_heatmap, cam_name = 'grad_cam_' + my_model_architecture +'_' + my_y_col + '_' + second_image + 'image_'+ image_number +'.jpg')

For refence, we can also take a look at the heatmaps on their own. The plot titles include the class predictions (we take the max as the final prediciton).  

In [ ]:
img_array = preprocess_input(get_img_array(first_img_path, size=img_size))
preds = model.predict(img_array)
plt.matshow(first_heatmap)
plt.title('Class Probs = ' + ''.join(str(x) for x in preds))
plt.show()

In [ ]:
img_array = preprocess_input(get_img_array(second_img_path, size=img_size))
preds = model.predict(img_array)
plt.matshow(second_heatmap)
plt.title('Class Probs = ' + ''.join(str(x) for x in preds))
plt.title('Class Probs = ' + ''.join(str(x) for x in preds))
plt.show()

# Visualizing Filter Layers  

Class Activation Maps showed us what parts of the image were important for the final class probabilities. We cannot make Class Activation maps for models trained on continuous outcomes because there are no "classes". Instead we can visualize the model layers when the model is generating a prediction on a image. This can help us get a feel for what features in that image the CNN model may be using to generate the prediction.  We can visualize any layer, but the deeper we go into the model, the more transformed the data is and the less interpretable it will be. Visualizing layers may not always provide clear information on what is going correctly with a model, but they can be helpful when the model is genereating strange predictions. It can provide clues when troubleshooting a model that does not meet your expectations.      


https://towardsdatascience.com/convolutional-neural-network-feature-map-and-filter-visualization-f75012a5a49c

https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/


In [ ]:
# get a list the available models, this will be a mix of pre-trained models and models you have trained today
available_models = os.listdir('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/')

# extract information from the model names in order to help undestand what the model was trained on
# it's imporant to note the "outcome" column, you can only select a model that predicts "cat" for CAMs
d = []
for x in available_models:
  d.append((x, re.findall('ufp_quint_cat|ufp_quint_cont|ufp', x)[0], re.findall('sat|str', x)[0], re.findall('xcep|resn|dens', x)[0], re.findall('model_(.*)e_lr', x)[0]))
available_models_df = pd.DataFrame(d, columns = ('model_name', 'outcome', 'type_of_image', 'model_arch', 'number_epochs'))
display(available_models_df)

In [ ]:
# run this code to pick and load a model
selected_model = int(input('Which model do you want to use for Class Activation Maps? Inidcate the row index (i.e., number) '))
print('You selected the ' + available_models_df.loc[selected_model, 'model_name'] + ' model')

# this loads the model
model = K.models.load_model('/content/drive/MyDrive/colab_files/sharp_d2_UFP/output/model/' + available_models_df.loc[selected_model, 'model_name'], custom_objects={'rmse': rmse})

# this sets the values
my_y_col, my_pred_col, my_type_of_images, my_model_architecture, my_number_of_epochs = available_models_df.loc[selected_model, 'outcome'], 'ufp_pred', available_models_df.loc[selected_model, 'type_of_image'], available_models_df.loc[selected_model, 'model_arch'], available_models_df.loc[selected_model, 'number_epochs']

if my_model_architecture == 'xcep':
  my_pre_pro_fn = K.applications.xception.preprocess_input
else:
  if my_model_architecture == 'resn':
    my_pre_pro_fn = K.applications.resnet_v2.preprocess_input
  else:
    if my_model_architecture == 'dens':
      my_pre_pro_fn = K.applications.densenet.preprocess_input

In [ ]:
# in case we need to reload the metadata
metadata = pd.read_csv("/content/drive/MyDrive/sharp_ml_course/pre_trained_models/ufp_colombia_metadata.csv")
if my_type_of_images == 'sat':
  metadata['train_file'] = metadata['sat_file']
else:
  metadata['train_file'] = metadata['str_file']

my_batch_size = 32
image_dir = '/content/images/'

In [ ]:
img_size = (240, 240)
preprocess_input = my_pre_pro_fn
img_path = '/content/images/images_angle1/01000.jpg' # 100 is a swimming pool

def get_img_array(img_path, size):
    img = K.preprocessing.image.load_img(img_path, target_size=size)
    array = K.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

display(Image(img_path))
img_array = preprocess_input(get_img_array(img_path, size=img_size))



In [ ]:
# visualizing every channel in every intermediate activation (or the first 6)

layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(img_array)

# only looking at first 6 layers, more than that can get confusing, plus layers after have 256 channels....which is a lot of visualize
sub_activations = activations[0:6]

layer_names = []
for layer in model.layers[0:16]:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot

images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    #plt.imsave('to_19_16028868_'+layer_name+'.png', display_grid, cmap='viridis')

Keep in mind that visualizing the filter layers probably won't be able to give us specific answers about when the CNN model is doing, but it might be able to provide some clues about what part of an image might be important with respect to the prediction generated. Also keep in mind that as you go deeper in the CNN (i.e. the later layers), the image will become progressively more transformed and less interpretable. You might see some activation maps that are all zero (i.e. all dark), which can indicate a dead filter. That is not necessarily a problem.

# END